In [0]:
import random
import time
import numpy as np
from random import choice, randint
import matplotlib.pyplot as plt

## GENETİK ALGORİTMA İLE YEMEK BULMA
##4.DENEME: 10*10'LUKTA YEMLER 2-10 ARASI DEĞERLER

##### KURALLAR:


###### -Birey giderken duvara çarparsa ölür


###### -Ortamdaki tüm yiyecekler yenildiğinde durur.


###### -Bir yiyecek üzerine gelindiğinde yenir.


###### -Yiyeceklerin nerede olduğunu bireyler bilmiyor.


YÖNLER: 1->SAĞ, 3->SOL, 2->YUKARI, 4->AŞAĞI

In [0]:
M=20#popülasyondaki her bireyin gen sayısı/çözüm uzunluğu 
birey_say=5
pop_size=(birey_say,M)#popülasyonun büyüklüğü belirlendi, nirey_say tane birey ve her bireyde M gen bulunuyor
k=5#yem miktarı belirlendi
N=10#kare dünyanın boyutu

In [0]:
#ilk populasyon randam olarak oluşturuldu,[1-4] arası sayılar
population=np.random.choice(np.arange(1,5,step=1),size=pop_size,replace=True)

In [0]:
for i in range(0,birey_say):
    print(str(i+1)+'. birey:',population[i])

1. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 4 3 3 4]
2. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 3 4 2 2 4 3 1]
3. birey: [3 2 1 4 4 1 2 4 2 2 4 2 2 3 4 2 2 4 3 1]
4. birey: [3 2 1 4 4 1 2 3 2 2 3 4 3 1 2 1 4 4 3 4]
5. birey: [3 2 1 4 4 1 2 1 3 4 4 1 2 3 4 2 2 4 3 1]


In [0]:
#bu fonksiyonda ortamın başlangıç durumu oluşturuluyor
def start():
    start_pos=[int(N/2),int(N/2)] #her birey N*N'lik kare dünyanın ortasından harekete başlar
    yemekler=[]#yemlerin yerleri 
    yemekler=np.random.choice(np.arange(2,(N-1),step=1),size=(k,2),replace=True)#k yem random olarak yerleştiriliyor-[1,N] arası değerler
    score=0#başlangıçta score değeri 0'dır.
    return start_pos,yemekler,score
    

In [0]:
#bu fonksiyonda oluşan yeni durumun yiyecek yiyip yemediği kontrol ediliyor
def check_eat_food(pos,yemler,direction,scr):
    if direction == 1: #sağ
        pos[0]+=1
        
    elif direction == 3: #sol
        pos[0] -= 1
        
    elif direction == 4: #aşağı
        pos[1] += 1
        
    else:
        pos[1] -= 1 #yukarı
    
    for i in range(0,(len(yemler))): #yem sayısı kadar döngüye girer 
        if (yemler[i] == pos).all():#yemin yeri ile oluşan yeni durum kontrol edilir
            scr=np.append(scr,(i+1))#indislerini ekliyoruz scr dizisine,0.indexle karışmasın diye 1 fazlasının
    return pos,scr

In [0]:
#burada ortamın sınırlarına çarpmasını kontrol ediyor
def collision_with_boundaries(position):
    if position[0] >= N or position[0] < 0 or position[1] >= N or position[1] < 0:
        return 1
    else:
        return 0

In [0]:
def fitness_fnc(pop_start,new_pop,fitness_degeri,yemek):
    yem_miktar=[]#yenilen yem miktarını tutacak olan dizi
    shpx=new_pop.shape[0]#birey sayısı
    shpy=new_pop.shape[1]#bireydeki çözüm/gen sayısı
    for i in range(0,shpx):
            print(str(i+1)+'. birey:',new_pop[i])
    for i in range(0,shpx):#her birey için dolaşan döngü
        birey_yem=np.zeros(len(yemek)) #her birey için yediği yemek miktarını 0'dan başlatıyor
        new_pos=[int(N/2),int(N/2)]#her birey ortamın ortasından harekete başlar   
        b_yemek=yemek#her birey için yemekler tekrardan ortama yerleştirilir 
        for j in range(0,shpy):#bireydeki her bir hareket icin girilen döngü
            new_pos, birey_yem= check_eat_food(new_pos,b_yemek,new_pop[i][j],birey_yem)#bireyin her bir yön değeri için yem yemesi kontrol ediliyor
            birey_yem=np.unique(birey_yem)#her yenenden bir tane alıyoruz, yani aynı yemi birden fazla yemesini önledik
            if collision_with_boundaries(new_pos)==1:#eğer birey sınırlara çarparsa ölür 
                new_pos=[int(N/2),int(N/2)]
                print(str(i+1)+". birey öldü....!!")
                birey_yem=np.zeros(len(yemek))
                birey_yem=np.unique(birey_yem)
                break
        print( str(i+1) +'. Bireyin uygunluk degeri:', len(birey_yem)-1)
        yem_miktar.append(len(birey_yem)-1)#0'lamıştık başta 0.indisi atmak için bunu -1 le topladık
    return np.array(yem_miktar)

In [0]:
#fittness değerine göre belirtilen sayıda bireyi sıralıyor max-to-min
def selection(pop, fitness, num_pop):
    parents = np.empty((num_pop, pop.shape[1]))#yeni popülasyon için en iyi durumlar parents adlı dizi oluşturuldu,pop_size büyüklüğünde
    for parent_num in range(num_pop):
        #fitness değeri en iyi olanı buluyoruz ve onun fitness değerini hiç seçilmeyecek bir değer atıyoruz, hep aynı durum/kromozom seçilmesin diye
        max_fitness_index = np.where(fitness == np.max(fitness))
        max_fitness_index = max_fitness_index[0][0]
        parents[parent_num, :] = pop[max_fitness_index, :]#fittness değeri max olanı seçiyoruzz
        fitness[max_fitness_index] = -99999999
    return parents

In [0]:
def crossover(old_pop, pop_size):
    cross_pop = np.empty(pop_size)#yeni bir popülasyon oluşturmak için popülasyon büyüklüğünde boş bir dizi oluşturduk
    cross_index = int(M/2)#crossover ortasından bölüyor bireyi
    for i in range(pop_size[0]):
        index1 = i%old_pop.shape[0]#old_pop'un içinden i. elemandan birey sayısına kadar olanları al
        index2 = (i+1)%old_pop.shape[0]#old_pop'un içinden i+1. elemandan birey sayısına kadar olanları al(0'da dahil etmek için birey sayısına bölümünden kalanla ifade edildi)
        cross_pop[i, 0:cross_index] = old_pop[index1, 0:cross_index]#önceki popülasyonun index1.bireyinin başından cross_index'e kadar cross_pop'un i.bireyine koy
        cross_pop[i, cross_index:] = old_pop[index2, cross_index:]#old_pop'un index2. bireyinin cross_index'ten sonuna olan genleri cross_pop'un i.bireyine koy
    return cross_pop

In [0]:
def mutation(cross_pop):    
    for i in range(cross_pop.shape[0]):#her birey için
        for j in range(3):#2 tane genini
            mutos_i = randint(0,cross_pop.shape[1]-1)#hangi indesin mutasyona ugrayacağı random karar veriliyor
            random_value = randint(1,4)#1-4 arası 
            cross_pop[i,mutos_i] = random_value
    
    return cross_pop

In [0]:
start,yemekler,score=start()
fitness=np.zeros(birey_say)
num_parent=randint(int(birey_say/2),birey_say)
i=0
max_yem=0

In [0]:
yemekler

array([[5, 6],
       [4, 3],
       [3, 5],
       [7, 4],
       [5, 3]])

In [0]:
while (max_yem!=k):
    print('##############   GENERATION ' + str(i+1)+ '  ###############' )
    
    # popülasyondaki bireylerin fitness değerleri ölçülüyor
    fitness = fitness_fnc(start,population,score,yemekler)
    max_yem=np.max(fitness)
    print(str(i+1)+'. jenerasyonda Max yenen yemek sayisi', np.max(fitness))
    
    parents = selection(population, fitness, num_parent)
    
    pop_crossover = crossover(parents, (pop_size[0] - parents.shape[0], M))

    pop_mutation = mutation(pop_crossover)

    population[0:parents.shape[0], :] = parents
    population[parents.shape[0]:, :] = pop_mutation
    i+=1

##############   GENERATION 1  ###############
1. birey: [1 1 1 4 2 3 1 1 2 1 4 3 4 2 2 3 1 2 1 1]
2. birey: [4 1 4 3 2 1 4 2 3 4 2 4 1 4 3 1 4 2 1 1]
3. birey: [3 2 1 4 4 1 2 1 2 2 3 4 2 3 2 1 3 2 1 1]
4. birey: [1 4 3 4 2 4 3 3 3 2 4 1 2 3 4 2 2 4 1 2]
5. birey: [4 4 2 3 3 2 4 1 1 3 1 1 1 1 1 3 1 3 1 4]
1. birey öldü....!!
1. Bireyin uygunluk degeri: 0
2. Bireyin uygunluk degeri: 1
3. Bireyin uygunluk degeri: 3
4. Bireyin uygunluk degeri: 2
5. Bireyin uygunluk degeri: 2
1. jenerasyonda Max yenen yemek sayisi 3
##############   GENERATION 2  ###############
1. birey: [3 2 1 4 4 1 2 1 2 2 3 4 2 3 2 1 3 2 1 1]
2. birey: [1 4 3 4 2 4 3 3 3 2 4 1 2 3 4 2 2 4 1 2]
3. birey: [3 2 1 4 3 1 2 1 2 2 4 4 2 3 4 2 2 4 1 2]
4. birey: [1 4 3 4 2 4 1 3 3 1 3 4 2 3 2 1 3 2 1 1]
5. birey: [3 2 1 4 4 1 2 1 2 2 4 1 2 3 4 2 2 4 4 4]
1. Bireyin uygunluk degeri: 3
2. Bireyin uygunluk degeri: 2
3. Bireyin uygunluk degeri: 1
4. Bireyin uygunluk degeri: 2
5. Bireyin uygunluk degeri: 2
2. jenerasyonda Max yenen

4. Bireyin uygunluk degeri: 1
5. Bireyin uygunluk degeri: 2
29. jenerasyonda Max yenen yemek sayisi 4
##############   GENERATION 30  ###############
1. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 3 4 2 2 4 3 1]
2. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 4 2 1 1]
3. birey: [3 4 1 4 4 1 2 1 2 2 3 4 2 3 2 1 4 2 1 1]
4. birey: [3 2 1 4 3 1 2 1 2 2 4 3 2 3 4 2 2 4 2 1]
5. birey: [3 2 1 4 4 1 4 1 2 2 3 4 3 2 2 3 4 2 1 1]
1. Bireyin uygunluk degeri: 4
2. Bireyin uygunluk degeri: 4
3. Bireyin uygunluk degeri: 2
4. Bireyin uygunluk degeri: 2
5. Bireyin uygunluk degeri: 1
30. jenerasyonda Max yenen yemek sayisi 4
##############   GENERATION 31  ###############
1. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 3 4 2 2 4 3 1]
2. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 4 2 1 1]
3. birey: [3 1 1 4 4 1 2 1 2 2 3 3 3 3 2 1 4 1 1 1]
4. birey: [3 2 1 4 4 1 3 1 2 2 1 3 2 3 4 2 2 4 3 1]
5. birey: [3 2 1 4 4 1 2 1 2 2 4 4 1 3 4 1 4 2 1 1]
1. Bireyin uygunluk degeri: 4
2. Bireyin uygunluk degeri: 4
3. Bireyin uygunluk degeri: 3


5. birey: [3 2 1 1 4 1 2 1 2 2 3 4 3 3 2 1 4 2 1 4]
1. Bireyin uygunluk degeri: 4
2. Bireyin uygunluk degeri: 4
3. Bireyin uygunluk degeri: 1
4. Bireyin uygunluk degeri: 2
5. Bireyin uygunluk degeri: 2
54. jenerasyonda Max yenen yemek sayisi 4
##############   GENERATION 55  ###############
1. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 3 4 2 2 4 3 1]
2. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 4 2 1 1]
3. birey: [1 2 1 4 4 1 2 1 4 4 3 4 3 3 2 1 4 2 1 1]
4. birey: [3 2 1 4 3 1 2 1 2 2 4 3 2 3 4 1 2 4 3 2]
5. birey: [3 2 1 3 4 1 2 1 4 2 3 4 3 3 2 1 1 2 1 1]
1. Bireyin uygunluk degeri: 4
2. Bireyin uygunluk degeri: 4
3. Bireyin uygunluk degeri: 1
4. Bireyin uygunluk degeri: 2
5. Bireyin uygunluk degeri: 2
55. jenerasyonda Max yenen yemek sayisi 4
##############   GENERATION 56  ###############
1. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 3 4 2 2 4 3 1]
2. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 4 2 1 1]
3. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 2 1 1 2]
4. birey: [3 2 1 4 4 1 2 1 2 2 4 3 1 3 4 2 2 3 2 1]


4. Bireyin uygunluk degeri: 2
5. Bireyin uygunluk degeri: 3
78. jenerasyonda Max yenen yemek sayisi 4
##############   GENERATION 79  ###############
1. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 3 4 2 2 4 3 1]
2. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 4 2 1 1]
3. birey: [3 2 1 4 4 1 2 3 2 1 3 4 3 3 2 1 4 2 1 1]
4. birey: [4 2 1 4 4 1 3 1 2 2 4 3 2 1 4 2 2 4 3 1]
5. birey: [3 2 1 4 4 1 2 3 2 2 3 4 3 3 2 3 4 2 1 1]
1. Bireyin uygunluk degeri: 4
2. Bireyin uygunluk degeri: 4
3. Bireyin uygunluk degeri: 2
4. Bireyin uygunluk degeri: 2
5. Bireyin uygunluk degeri: 3
79. jenerasyonda Max yenen yemek sayisi 4
##############   GENERATION 80  ###############
1. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 3 4 2 2 4 3 1]
2. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 4 2 1 1]
3. birey: [3 2 1 4 4 1 2 2 2 2 3 1 3 2 2 1 4 2 1 1]
4. birey: [3 2 1 4 4 1 2 1 4 2 4 3 2 3 4 2 2 4 3 1]
5. birey: [3 2 1 4 4 4 2 2 2 2 3 4 3 3 2 1 2 2 1 1]
1. Bireyin uygunluk degeri: 4
2. Bireyin uygunluk degeri: 4
3. Bireyin uygunluk degeri: 1


5. birey: [3 2 1 4 4 1 2 1 2 3 3 4 3 2 2 1 4 2 1 1]
1. Bireyin uygunluk degeri: 4
2. Bireyin uygunluk degeri: 4
3. Bireyin uygunluk degeri: 4
4. Bireyin uygunluk degeri: 4
5. Bireyin uygunluk degeri: 4
97. jenerasyonda Max yenen yemek sayisi 4
##############   GENERATION 98  ###############
1. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 3 4 2 2 4 3 1]
2. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 4 2 1 1]
3. birey: [3 2 1 4 4 1 4 1 2 2 3 4 3 3 2 4 3 2 1 1]
4. birey: [3 2 1 4 4 2 2 1 2 2 4 3 2 3 4 2 2 4 3 1]
5. birey: [3 2 1 4 4 2 2 1 2 3 3 4 3 3 2 1 4 2 2 1]
1. Bireyin uygunluk degeri: 4
2. Bireyin uygunluk degeri: 4
3. Bireyin uygunluk degeri: 2
4. Bireyin uygunluk degeri: 3
5. Bireyin uygunluk degeri: 3
98. jenerasyonda Max yenen yemek sayisi 4
##############   GENERATION 99  ###############
1. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 3 4 2 2 4 3 1]
2. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 4 2 1 1]
3. birey: [2 4 1 4 4 1 1 1 2 2 3 4 3 3 2 1 4 2 1 1]
4. birey: [3 2 1 4 4 1 2 1 2 3 4 3 2 3 4 2 2 4 1 3]


...

2. Bireyin uygunluk degeri: 4
3. Bireyin uygunluk degeri: 1
4. Bireyin uygunluk degeri: 3
5. Bireyin uygunluk degeri: 2
190. jenerasyonda Max yenen yemek sayisi 4
##############   GENERATION 191  ###############
1. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 3 4 2 2 4 3 1]
2. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 4 2 1 1]
3. birey: [3 2 1 4 1 1 1 1 2 2 3 4 3 3 2 1 4 2 1 1]
4. birey: [3 1 1 4 2 1 2 1 2 2 2 3 2 3 4 2 2 4 3 1]
5. birey: [3 2 1 4 4 1 2 1 2 1 3 4 3 3 2 1 3 2 1 1]
1. Bireyin uygunluk degeri: 4
2. Bireyin uygunluk degeri: 4
3. Bireyin uygunluk degeri: 1
4. birey öldü....!!
4. Bireyin uygunluk degeri: 0
5. Bireyin uygunluk degeri: 3
191. jenerasyonda Max yenen yemek sayisi 4
##############   GENERATION 192  ###############
1. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 3 4 2 2 4 3 1]
2. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 4 2 1 1]
3. birey: [3 1 1 4 4 1 2 1 1 2 3 4 3 4 2 1 4 2 1 1]
4. birey: [3 2 1 4 4 1 2 1 2 3 4 3 2 3 4 3 2 4 3 1]
5. birey: [3 2 1 4 4 3 2 1 2 2 3 4 3 2 1 1 4 2 1 1]
1. Bir

2. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 4 2 1 1]
3. birey: [3 3 1 4 4 1 2 1 2 2 3 4 3 1 2 1 4 2 2 1]
4. birey: [3 2 1 4 4 1 2 1 2 3 4 3 1 3 4 2 2 4 3 1]
5. birey: [3 2 2 4 4 1 2 3 2 2 3 4 3 3 2 4 4 2 1 1]
1. Bireyin uygunluk degeri: 4
2. Bireyin uygunluk degeri: 4
3. Bireyin uygunluk degeri: 2
4. Bireyin uygunluk degeri: 2
5. Bireyin uygunluk degeri: 1
217. jenerasyonda Max yenen yemek sayisi 4
##############   GENERATION 218  ###############
1. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 3 4 2 2 4 3 1]
2. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 4 2 1 1]
3. birey: [4 2 1 3 4 1 2 1 2 2 3 4 3 3 2 1 2 2 1 1]
4. birey: [3 2 4 4 4 1 2 1 2 2 4 3 2 3 4 2 1 4 2 1]
5. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 2 2 1 1]
1. Bireyin uygunluk degeri: 4
2. Bireyin uygunluk degeri: 4
3. Bireyin uygunluk degeri: 4
4. Bireyin uygunluk degeri: 1
5. Bireyin uygunluk degeri: 4
218. jenerasyonda Max yenen yemek sayisi 4
##############   GENERATION 219  ###############
1. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 3 4 2 2 4 3

1. Bireyin uygunluk degeri: 4
2. Bireyin uygunluk degeri: 4
3. Bireyin uygunluk degeri: 3
4. Bireyin uygunluk degeri: 1
5. Bireyin uygunluk degeri: 4
235. jenerasyonda Max yenen yemek sayisi 4
##############   GENERATION 236  ###############
1. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 3 4 2 2 4 3 1]
2. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 4 2 1 1]
3. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 4 2 2 1]
4. birey: [1 2 1 4 4 1 2 1 2 2 4 3 4 3 4 2 3 4 3 1]
5. birey: [3 2 1 4 4 1 2 1 3 2 3 4 3 1 2 3 4 2 1 1]
1. Bireyin uygunluk degeri: 4
2. Bireyin uygunluk degeri: 4
3. Bireyin uygunluk degeri: 4
4. Bireyin uygunluk degeri: 2
5. Bireyin uygunluk degeri: 1
236. jenerasyonda Max yenen yemek sayisi 4
##############   GENERATION 237  ###############
1. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 3 4 2 2 4 3 1]
2. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 4 2 1 1]
3. birey: [3 2 1 4 4 2 2 1 2 2 3 4 3 3 3 1 4 3 1 1]
4. birey: [3 1 1 4 4 1 2 1 3 2 4 3 2 3 4 2 1 4 3 1]
5. birey: [3 2 1 4 3 2 2 1 2 4 3 4 3 3 2 1 4 2 1

##############   GENERATION 270  ###############
1. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 3 4 2 2 4 3 1]
2. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 4 2 1 1]
3. birey: [4 2 1 4 4 1 2 1 2 2 3 4 3 3 2 3 4 3 1 1]
4. birey: [4 2 1 4 4 1 2 1 4 2 4 3 2 3 4 2 2 4 3 1]
5. birey: [3 2 1 4 4 1 2 1 4 2 3 3 3 3 2 1 4 2 1 1]
1. Bireyin uygunluk degeri: 4
2. Bireyin uygunluk degeri: 4
3. Bireyin uygunluk degeri: 3
4. Bireyin uygunluk degeri: 1
5. Bireyin uygunluk degeri: 2
270. jenerasyonda Max yenen yemek sayisi 4
##############   GENERATION 271  ###############
1. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 3 4 2 2 4 3 1]
2. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 4 2 1 1]
3. birey: [3 2 1 4 4 4 2 1 2 2 3 1 3 3 2 1 4 2 1 1]
4. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 4 3 2 2 4 3 1]
5. birey: [3 2 1 4 4 1 2 1 2 2 3 4 2 2 2 1 4 2 1 1]
1. Bireyin uygunluk degeri: 4
2. Bireyin uygunluk degeri: 4
3. Bireyin uygunluk degeri: 3
4. Bireyin uygunluk degeri: 4
5. Bireyin uygunluk degeri: 2
271. jenerasyonda Max yenen yemek sayis

1. Bireyin uygunluk degeri: 4
2. Bireyin uygunluk degeri: 4
3. Bireyin uygunluk degeri: 1
4. Bireyin uygunluk degeri: 4
5. Bireyin uygunluk degeri: 2
291. jenerasyonda Max yenen yemek sayisi 4
##############   GENERATION 292  ###############
1. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 3 4 2 2 4 3 1]
2. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 4 2 1 1]
3. birey: [3 2 1 4 4 1 2 3 2 2 3 4 3 3 1 1 4 1 1 1]
4. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 3 4 2 2 4 3 2]
5. birey: [3 1 1 4 1 1 2 1 2 2 3 4 3 3 2 1 4 2 1 2]
1. Bireyin uygunluk degeri: 4
2. Bireyin uygunluk degeri: 4
3. Bireyin uygunluk degeri: 3
4. Bireyin uygunluk degeri: 4
5. Bireyin uygunluk degeri: 1
292. jenerasyonda Max yenen yemek sayisi 4
##############   GENERATION 293  ###############
1. birey: [3 2 1 4 4 1 2 1 2 2 4 3 2 3 4 2 2 4 3 1]
2. birey: [3 2 1 4 4 1 2 1 2 2 3 4 3 3 2 1 4 2 1 1]
3. birey: [2 2 1 4 2 1 2 1 4 2 3 4 3 3 2 1 4 2 1 1]
4. birey: [3 2 1 1 4 1 1 1 2 2 4 3 2 3 4 2 3 4 3 1]
5. birey: [3 2 3 4 4 1 2 1 2 2 3 4 3 2 2 1 4 2 1